In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

## client profile data


In [ ]:
df_client = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_demo.txt')

In [ ]:
df_client.head(2)
df_client.columns 
df_client.shape 

#### clean column names

In [ ]:

df_client.rename(columns={'gendr':'gender',
                          'clnt_tenure_yr':'client_tenure_year',
                          'clnt_tenure_mnth':'client_tenure_month',
                          'clnt_age':'client_age',
                          'num_accts':'number_of_accounts',
                          'calls_6_mnth':'calls_6_month',
                          'logons_6_mnth':'logos_6_month',
                          'bal':'balance'},inplace=True)


#### handle null values

In [ ]:
df_client.isna().sum()

display(df_client[df_client.isna().any(axis=1)])

In [ ]:
# for client_id = 4666211/index = 9583, fill age with mean_age
df_client.at[9583, 'client_age'] = df_client['client_age'].mean() 


# for the rest, fill with -1 if numeric variable or 'unknown' if categorical variable
df_client.fillna({
    'client_tenure_year': -1,
    'client_tenure_month': -1,
    'client_age':-1,
    'gender':'unknown',
    'number_of_accounts':-1,
    'balance':-1,
    'calls_6_month':-1,
    'logos_6_month':-1,},inplace=True)

df_client.isna().sum()

#### column gender-> replace 'X' and 'U' with 'unknown', create new column: gender_cleaned

In [ ]:
df_client['gender'].value_counts()

In [ ]:
df_client['gender_cleaned'] = df_client['gender'].replace({'X':'unknown','U':'unknown'})
df_client['gender_cleaned'].value_counts()

#### univariate analysis

In [ ]:
df_client.nunique()

In [ ]:
# barplot --->  categorical variable + numeric discrete variables 

categorical_cols = ['gender_cleaned', 'client_tenure_year', 'number_of_accounts','calls_6_month','logos_6_month']

fig, axes = plt.subplots(nrows=len(categorical_cols), ncols=1, figsize=(20, 4*len(categorical_cols)))

for i, col in enumerate(categorical_cols):
    sns.countplot(data=df_client, x=col, ax=axes[i])
    axes[i].set_title(f"Distribution of {col}")
    axes[i].set_ylabel("Count")
    axes[i].set_xlabel(col)

plt.tight_layout()
plt.show()


In [ ]:
# histogram --->  numeric continuous variables , numeric_cols = ['client_tenure_month','client_age','balance']

# fixed_sized binning
sns.histplot(df_client['client_age'],bins=30) 
plt.show()


In [ ]:
# quantile binning
p95_balance = df_client['balance'].quantile(0.95)

balance_below_95 = df_client[df_client['balance'] <= p95_balance]['balance']
balance_below_95

bin_edges = np.linspace(balance_below_95.min(),balance_below_95.max(),30)

sns.histplot(df_client['balance'],bins=bin_edges )  

plt.show()


In [ ]:
# quantile binning 
p95_month = df_client['client_tenure_month'].quantile(0.95)

balance_below_95 = df_client[df_client['client_tenure_month'] <= p95_month]['client_tenure_month']
balance_below_95

bin_edges = np.linspace(balance_below_95.min(),balance_below_95.max(),30)

sns.histplot(df_client['client_tenure_month'],bins=bin_edges )  

plt.show()


#### bivariate analysis

In [ ]:
# age - tenure year relationship
df_primary_client = df_client[(df_client['client_age']>=20)& (df_client['client_age'] <= 60)]
sns.scatterplot(data = df_primary_client, x = 'client_age',y = 'client_tenure_year')
plt.show()

## Digital Footprints data

In [ ]:
df_footprints_1 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_1.txt')
df_footprints_2 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_2.txt')

df_footprints_1.columns, df_footprints_2.columns

In [ ]:
df_footprints_1.dtypes, df_footprints_2.dtypes

#### change dtype for date_time column 

In [ ]:
# date_time column, object dtype ---> datetime dtype(each value is datetime object)
df_footprints_1['date_time'] = pd.to_datetime(df_footprints_1['date_time'],errors = 'coerce')
df_footprints_1['date_time'].isna().sum()
df_footprints_2['date_time'] = pd.to_datetime(df_footprints_2['date_time'],errors = 'coerce')
df_footprints_2['date_time'].isna().sum()

df_footprints_1.dtypes, df_footprints_2.dtypes

#### drop duplicate rows in 2 datasets 

In [ ]:
df_footprints_1.duplicated().sum(), df_footprints_2.duplicated().sum()

In [ ]:
df_footprints_1.drop_duplicates(inplace=True)
df_footprints_2.drop_duplicates(inplace=True)
df_footprints_1.duplicated().sum(), df_footprints_2.duplicated().sum()

#### concat two dataframes

In [ ]:
df_footprints_combined = pd.concat([df_footprints_1,df_footprints_2],ignore_index=True)

df_footprints_combined.shape, df_footprints_combined.duplicated().sum()

In [ ]:
df_footprints_combined.nunique()

#### explore date time 

In [ ]:
df_footprints_combined['date_time'].min(),df_footprints_combined['date_time'].max()

In [ ]:
df_footprints_combined.tail(3)

In [ ]:
display(df_footprints_combined[df_footprints_combined['client_id'] == 9668240].sort_values(by=["client_id","visit_id","date_time"]))
df_footprints_combined[df_footprints_combined['client_id'] == 9668240].sort_values(by=["client_id","visit_id","date_time"]).groupby(["client_id","visit_id"])["date_time"].diff()

## Experiment data

In [ ]:

df_experiments = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_experiment_clients.txt')

df_experiments.columns, df_experiments.shape,df_experiments.dtypes


(Index(['client_id', 'Variation'], dtype='object'),
 (70609, 2),
 client_id     int64
 Variation    object
 dtype: object)

In [ ]:
 # rename column Variation-> variation
df_experiments.rename(columns={'Variation':'variation'},inplace=True)

In [ ]:
# handle null values in column 'Variation', fill with 'unknown'
df_experiments.isna().sum()
df_experiments['variation_cleaned'] = df_experiments['variation'].fillna('unknown')

In [ ]:
df_experiments['variation_cleaned'].value_counts()

variation_cleaned
Test       26968
Control    23532
unknown    20109
Name: count, dtype: int64

## combine all data sources, df_final (744641, 14), use gender_clean, variation_clean

In [ ]:
# digital footprint is main table , merge 3 datasets

df1= pd.merge(left=df_footprints_combined,right=df_experiments,how='left', on='client_id')
df_final = pd.merge(left = df1,right=df_client, how='left',on = 'client_id')

In [ ]:
df_final.shape, df_final.isna().sum()


((744641, 16),
 client_id                   0
 visitor_id                  0
 visit_id                    0
 process_step                0
 date_time                   0
 variation              427406
 variation_cleaned      300744
 client_tenure_year     300744
 client_tenure_month    300744
 client_age             300744
 gender                 300744
 number_of_accounts     300744
 balance                300744
 calls_6_month          300744
 logos_6_month          300744
 gender_cleaned         300744
 dtype: int64)

In [ ]:
display(df_final[df_final.isna().any(axis=1)])

In [ ]:
# fill all null values (clients that are in footprint but not in in client profile) 

def handle_null(col):
    if np.issubdtype(col.dtype, np.number):  # float64 columns
        return col.fillna(-1)
    elif col.dtype.kind == 'O':
        return col.fillna('unknown')
    else:
        return col

df_final = df_final.apply(handle_null)

df_final.isna().sum()

client_id              0
visitor_id             0
visit_id               0
process_step           0
date_time              0
variation              0
variation_cleaned      0
client_tenure_year     0
client_tenure_month    0
client_age             0
gender                 0
number_of_accounts     0
balance                0
calls_6_month          0
logos_6_month          0
gender_cleaned         0
dtype: int64

In [ ]:
df_final['variation_cleaned'].value_counts()

variation_cleaned
unknown    427406
Test       176699
Control    140536
Name: count, dtype: int64